In [11]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam,SGD,RMSprop,Adadelta
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
main_path_energy = r"C:\Users\SEBASTIAN\Documents\GitHub\problemaEspecialSalo\drone_irrigation\src\data_base\paths_energy/"
main_path_total_energy = r"C:\Users\SEBASTIAN\Documents\GitHub\problemaEspecialSalo\drone_irrigation\src\data_base\paths_totals/"

energy_path11_pd = pd.read_csv(main_path_energy + 'feat_pathv11_nuevo.csv')
energy_path12_pd = pd.read_csv(main_path_energy + 'feat_pathv12_nuevo.csv')
energy_path13_pd = pd.read_csv(main_path_energy + 'features_pathv13.csv')

energy_path_list = [energy_path11_pd,energy_path12_pd,energy_path13_pd]
complete_energy_pd = pd.concat(energy_path_list)

total_path11_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv11.csv')
total_path12_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv12_nuevo.csv')
total_path13_pd = pd.read_csv(main_path_total_energy + 'totals_pathv13.csv')

total_path_list = [total_path11_pd,total_path12_pd,total_path13_pd]
complete_total_pd = pd.concat(total_path_list)

In [3]:
eps = 5
complete_energy_pd = complete_energy_pd[(complete_energy_pd['sim_point_dist']-complete_energy_pd['teo_point_dist']) < eps]
print(f'new shape of complete_energy_pd = {complete_energy_pd.shape}')

new shape of complete_energy_pd = (2577, 14)


In [4]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler2 = StandardScaler()

x_train = scaler2.fit_transform(x_train)
x_test = scaler2.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (2061, 516)


#### Pruebas con Optimizer: Adam,variando lr

In [10]:
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=5,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_Adam = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adam(learning_rate=i), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_Adam[0,idx] = r2
    results_Adam[1,idx] = RMSE
    results_Adam[2,idx] = MSE
    results_Adam[3,idx] = model

print(results_Adam)

Epoch 1/250
52/52 [==============================] - 2s 18ms/step - loss: 24.2929 - mean_squared_logarithmic_error: 24.2929 - val_loss: 0.1292 - val_mean_squared_logarithmic_error: 0.1292
Epoch 2/250
52/52 [==============================] - 0s 9ms/step - loss: 0.1342 - mean_squared_logarithmic_error: 0.1342 - val_loss: 0.1133 - val_mean_squared_logarithmic_error: 0.1133
Epoch 3/250
52/52 [==============================] - 0s 9ms/step - loss: 0.1265 - mean_squared_logarithmic_error: 0.1265 - val_loss: 0.1222 - val_mean_squared_logarithmic_error: 0.1222
Epoch 4/250
52/52 [==============================] - 0s 9ms/step - loss: 0.1300 - mean_squared_logarithmic_error: 0.1300 - val_loss: 0.1115 - val_mean_squared_logarithmic_error: 0.1115
Epoch 5/250
52/52 [==============================] - 1s 10ms/step - loss: 0.1206 - mean_squared_logarithmic_error: 0.1206 - val_loss: 0.1183 - val_mean_squared_logarithmic_error: 0.1183
Epoch 6/250
52/52 [==============================] - 0s 9ms/step - loss

In [12]:
print(results_Adam)

[[0.72153770991692 0.7297424124341261 0.7296434250748736
  0.7293914471681329 0.729341461341717]
 [755483.2678197859 744270.1525152995 744406.4421062332 744753.2623967133
  744822.0432758689]
 [570754967955.6622 553938059925.1472 554140951049.2607 554657421850.5476
  554759876149.6403]


#### SGD with momentum, variando el lr y dejando el momemtum en 0.9

In [16]:
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=5,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_SGD = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=SGD(learning_rate=i,momentum=0.9), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_SGD[0,idx] = r2
    results_SGD[1,idx] = RMSE
    results_SGD[2,idx] = MSE
    results_SGD[3,idx] = model


In [17]:
print(results_SGD)

[[0.7345724252974902 0.7351068087745087 0.7351673423396645
  0.7351571885178596 0.7351558052353719]
 [737589.4128363254 736846.5469469804 736762.3498093055 736776.4735997285
  736778.397700777]
 [544038141928.2352 542942833747.68866 542818760096.5295
  542839572050.0516 542842407318.52435]


#### RMSprop variando el learning rate, momentum 0.9

In [18]:
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=5,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_RMSprop = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=RMSprop(learning_rate=i,momentum=0.9), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_RMSprop[0,idx] = r2
    results_RMSprop[1,idx] = RMSE
    results_RMSprop[2,idx] = MSE
    results_RMSprop[3,idx] = model


In [19]:
print(results_RMSprop)

[[-19895903.44863128 -19895903.44863128 -19895903.44863128
  -19895903.44863128 -19895903.44863128]
 [6385920362.112925 6385920362.112925 6385920362.112925 6385920362.112925
  6385920362.112925]
 [4.077997887124846e+19 4.077997887124846e+19 4.077997887124846e+19
  4.077997887124846e+19 4.077997887124846e+19]


#### Adadelta, variando el lr

In [20]:
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=5,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_Adadel = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adadelta(learning_rate=i), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_Adadel[0,idx] = r2
    results_Adadel[1,idx] = RMSE
    results_Adadel[2,idx] = MSE
    results_Adadel[3,idx] = model

In [21]:
print(results_Adadel)

[[0.7366645729906165 0.7365839456829555 0.7365687916445584
  0.7365640619017217 0.7365639644352445]
 [734676.7560181023 734789.2180519863 734810.3535592946 734816.950060195
  734817.0859945961]
 [539749935833.2822 539915194965.4494 539946255697.9356 539955950095.7672
  539956149869.5896]


In [23]:
results_list = [results_Adam,results_Adadel,results_RMSprop,results_SGD]
results = [i[0,:] for i in results_list]
print(np.max(results))

0.7366645729906165


In [24]:
results

[array([0.72153770991692, 0.7297424124341261, 0.7296434250748736,
        0.7293914471681329, 0.729341461341717], dtype=object),
 array([0.7366645729906165, 0.7365839456829555, 0.7365687916445584,
        0.7365640619017217, 0.7365639644352445], dtype=object),
 array([-19895903.44863128, -19895903.44863128, -19895903.44863128,
        -19895903.44863128, -19895903.44863128], dtype=object),
 array([0.7345724252974902, 0.7351068087745087, 0.7351673423396645,
        0.7351571885178596, 0.7351558052353719], dtype=object)]